# Spark SQL

This notebook introduces how to perform basic data transformation and exploration using Spark SQL on [Avocado Prices dataset](https://www.kaggle.com/datasets/neuromusic/avocado-prices) in Kaggle.

## Spark Cluster Preparation

In [1]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [2]:
if IN_COLAB:
    !apt-get install openjdk-8-jdk-headless -qq > /dev/null
    !wget -q https://dlcdn.apache.org/spark/spark-3.5.5/spark-3.5.5-bin-hadoop3.tgz
    !tar xf spark-3.5.5-bin-hadoop3.tgz
    !mv spark-3.5.5-bin-hadoop3 spark
    !pip install -q findspark
    import os
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
    os.environ["SPARK_HOME"] = "/content/spark"

In [3]:
import findspark
findspark.init()

In [4]:
spark_url = 'local'

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [6]:
spark = SparkSession.builder\
        .master(spark_url)\
        .appName('Spark SQL')\
        .getOrCreate()

## Spark SQL Data Preparation

First, we read a csv file.  We can provide option such as delimiter and header.  We then rename the colume names to remove dot ('.') in the names.

In [7]:
path = 'avocado.csv'

In [8]:
df = spark.read.csv(path, header=True, inferSchema=True)

Check out data and schema

In [9]:
df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- AveragePrice: double (nullable = true)
 |-- Total Volume: double (nullable = true)
 |-- 4046: double (nullable = true)
 |-- 4225: double (nullable = true)
 |-- 4770: double (nullable = true)
 |-- Total Bags: double (nullable = true)
 |-- Small Bags: double (nullable = true)
 |-- Large Bags: double (nullable = true)
 |-- XLarge Bags: double (nullable = true)
 |-- type: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- region: string (nullable = true)



Those columns with spaces or dots in the column names will be a bit difficult for Spark SQL, esepcially when we want to filter data.  We must first remove spaces/dots in the column names.  This is optional, though.

In [10]:
cols = [c.replace(' ', '_') for c in df.columns]
df = df.toDF(*cols)

In [11]:
df.columns

['Id',
 'Date',
 'AveragePrice',
 'Total_Volume',
 '4046',
 '4225',
 '4770',
 'Total_Bags',
 'Small_Bags',
 'Large_Bags',
 'XLarge_Bags',
 'type',
 'year',
 'region']

In [12]:
df.show(5)

+---+----------+------------+------------+-------+---------+-----+----------+----------+----------+-----------+------------+----+------+
| Id|      Date|AveragePrice|Total_Volume|   4046|     4225| 4770|Total_Bags|Small_Bags|Large_Bags|XLarge_Bags|        type|year|region|
+---+----------+------------+------------+-------+---------+-----+----------+----------+----------+-----------+------------+----+------+
|  0|2015-12-27|        1.33|    64236.62|1036.74| 54454.85|48.16|   8696.87|   8603.62|     93.25|        0.0|conventional|2015|Albany|
|  1|2015-12-20|        1.35|    54876.98| 674.28| 44638.81|58.33|   9505.56|   9408.07|     97.49|        0.0|conventional|2015|Albany|
|  2|2015-12-13|        0.93|   118220.22|  794.7|109149.67|130.5|   8145.35|   8042.21|    103.14|        0.0|conventional|2015|Albany|
|  3|2015-12-06|        1.08|    78992.15| 1132.0| 71976.41|72.58|   5811.16|    5677.4|    133.76|        0.0|conventional|2015|Albany|
|  4|2015-11-29|        1.28|     51039.6

Using *'inferSchema=True'*, Spark SQL seems to guess datatype correctly.  However, if it is incorrect, we can cast each column to proper type using **'cast'** and replace back to the same column using **'withColumn'**.

## Basic Spark SQL Commands

We can select some columns using **'select'** and select some rows using **'filter'**.  Note that we can perform basic math to columns.

In [13]:
df.select(df['Date'], df['AveragePrice'], df['Total_Bags'], df['year'], df['region']).show(5)

+----------+------------+----------+----+------+
|      Date|AveragePrice|Total_Bags|year|region|
+----------+------------+----------+----+------+
|2015-12-27|        1.33|   8696.87|2015|Albany|
|2015-12-20|        1.35|   9505.56|2015|Albany|
|2015-12-13|        0.93|   8145.35|2015|Albany|
|2015-12-06|        1.08|   5811.16|2015|Albany|
|2015-11-29|        1.28|   6183.95|2015|Albany|
+----------+------------+----------+----+------+
only showing top 5 rows



In [14]:
df.select(df['Date'], df['Small_Bags']+df['Large_Bags']+df['XLarge_Bags'],
          df['Total_Bags']).show(5)

+----------+-----------------------------------------+----------+
|      Date|((Small_Bags + Large_Bags) + XLarge_Bags)|Total_Bags|
+----------+-----------------------------------------+----------+
|2015-12-27|                                  8696.87|   8696.87|
|2015-12-20|                                  9505.56|   9505.56|
|2015-12-13|                                  8145.35|   8145.35|
|2015-12-06|                                  5811.16|   5811.16|
|2015-11-29|                                  6183.95|   6183.95|
+----------+-----------------------------------------+----------+
only showing top 5 rows



In [15]:
df.select(df['Date'], df['Total_Bags'], df['Total_Volume'],
          df['Total_Volume']/df['Total_Bags']).show(5)

+----------+----------+------------+---------------------------+
|      Date|Total_Bags|Total_Volume|(Total_Volume / Total_Bags)|
+----------+----------+------------+---------------------------+
|2015-12-27|   8696.87|    64236.62|          7.386176865929926|
|2015-12-20|   9505.56|    54876.98|          5.773145401217814|
|2015-12-13|   8145.35|   118220.22|         14.513829362765259|
|2015-12-06|   5811.16|    78992.15|         13.593181051631687|
|2015-11-29|   6183.95|     51039.6|          8.253559618043484|
+----------+----------+------------+---------------------------+
only showing top 5 rows



In [16]:
df.filter(df['Total_Bags'] < 8000).show(3)

+---+----------+------------+------------+-------+--------+-----+----------+----------+----------+-----------+------------+----+------+
| Id|      Date|AveragePrice|Total_Volume|   4046|    4225| 4770|Total_Bags|Small_Bags|Large_Bags|XLarge_Bags|        type|year|region|
+---+----------+------------+------------+-------+--------+-----+----------+----------+----------+-----------+------------+----+------+
|  3|2015-12-06|        1.08|    78992.15| 1132.0|71976.41|72.58|   5811.16|    5677.4|    133.76|        0.0|conventional|2015|Albany|
|  4|2015-11-29|        1.28|     51039.6| 941.48|43838.39|75.78|   6183.95|   5986.26|    197.69|        0.0|conventional|2015|Albany|
|  5|2015-11-22|        1.26|    55979.78|1184.27|48067.99|43.61|   6683.91|   6556.47|    127.44|        0.0|conventional|2015|Albany|
+---+----------+------------+------------+-------+--------+-----+----------+----------+----------+-----------+------------+----+------+
only showing top 3 rows



In [17]:
df.filter((df['Total_Bags'] < 8000) & (df.year > 2015)).select('Date', 'Total_Bags').show(3)

+----------+----------+
|      Date|Total_Bags|
+----------+----------+
|2016-12-04|   6704.29|
|2016-12-04|   7088.23|
|2016-10-30|   6794.05|
+----------+----------+
only showing top 3 rows



In [18]:
df.filter('Total_Bags < 8000 and year > 2015').select('Date', 'Total_Bags').show(3)

+----------+----------+
|      Date|Total_Bags|
+----------+----------+
|2016-12-04|   6704.29|
|2016-12-04|   7088.23|
|2016-10-30|   6794.05|
+----------+----------+
only showing top 3 rows



In [19]:
df.filter('region == "SanDiego"').show(3)

+---+----------+------------+------------+---------+---------+--------+----------+----------+----------+-----------+------------+----+--------+
| Id|      Date|AveragePrice|Total_Volume|     4046|     4225|    4770|Total_Bags|Small_Bags|Large_Bags|XLarge_Bags|        type|year|  region|
+---+----------+------------+------------+---------+---------+--------+----------+----------+----------+-----------+------------+----+--------+
|  0|2015-12-27|        0.92|    439968.4|141447.28|130341.75|20174.55| 148004.82| 116383.07|  31621.75|        0.0|conventional|2015|SanDiego|
|  1|2015-12-20|        0.94|   420476.44|130565.46| 119298.5|25052.93| 145559.55| 111019.22|  34540.33|        0.0|conventional|2015|SanDiego|
|  2|2015-12-13|        0.84|    462548.3|155111.21|118664.89|16726.87| 172045.33| 128053.44|  43991.89|        0.0|conventional|2015|SanDiego|
+---+----------+------------+------------+---------+---------+--------+----------+----------+----------+-----------+------------+----+--

## Aggregate and Groupby Functions
We can use several built-in aggegrate functions.  We can also use groupby for group operations

### Explore Distinct Values

Let's explore those categorical data including region and type using select-distinct.  Note that it can be applied to those numerical data too

In [21]:
df.select('type').distinct().show()

+------------+
|        type|
+------------+
|     organic|
|conventional|
+------------+



In [22]:
df.select('region').distinct().show()

+------------------+
|            region|
+------------------+
|     PhoenixTucson|
|       GrandRapids|
|     SouthCarolina|
|           TotalUS|
|  WestTexNewMexico|
|        Louisville|
|      Philadelphia|
|        Sacramento|
|     DallasFtWorth|
|      Indianapolis|
|          LasVegas|
|         Nashville|
|        GreatLakes|
|           Detroit|
|            Albany|
|          Portland|
|  CincinnatiDayton|
|          SanDiego|
|             Boise|
|HarrisburgScranton|
+------------------+
only showing top 20 rows



In [23]:
df.select('year').distinct().show()

+----+
|year|
+----+
|2018|
|2015|
|2016|
|2017|
+----+



### Using group and groupby functions

In [24]:
from pyspark.sql.functions import avg, min, max

In [25]:
df.select(min('AveragePrice'), avg('AveragePrice'), max('AveragePrice')).show()

+-----------------+------------------+-----------------+
|min(AveragePrice)| avg(AveragePrice)|max(AveragePrice)|
+-----------------+------------------+-----------------+
|             0.44|1.4059784097758825|             3.25|
+-----------------+------------------+-----------------+



In [26]:
df.filter('region == "SanDiego"').select(avg('AveragePrice')).show()

+------------------+
| avg(AveragePrice)|
+------------------+
|1.3981656804733738|
+------------------+



Groupby function allows us to work data in groups.

In [27]:
df.groupby('type').count().show()

+------------+-----+
|        type|count|
+------------+-----+
|     organic| 9123|
|conventional| 9126|
+------------+-----+



In [28]:
df.groupby('year', 'type').agg({'AveragePrice': 'avg'}).orderBy('year', 'type').show()

+----+------------+------------------+
|year|        type| avg(AveragePrice)|
+----+------------+------------------+
|2015|conventional|1.0779629629629648|
|2015|     organic|1.6733238332739595|
|2016|conventional|1.1055947293447288|
|2016|     organic|1.5716844729344717|
|2017|conventional|1.2948881900768692|
|2017|     organic|1.7355209790209802|
|2018|conventional| 1.127885802469136|
|2018|     organic| 1.567175925925925|
+----+------------+------------------+



## User-Defined Function
We can create user-defined function using udf.

In [29]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

In [30]:
def pricegroup_mapping(price):
    if price < 1:
        return 'cheap'
    if price < 2:
        return 'moderate'
    return 'expensive'

to_pricegroup = udf(pricegroup_mapping, StringType())

In [31]:
df.select('Date', 'AveragePrice', to_pricegroup('AveragePrice')).show(5)

+----------+------------+--------------------------------+
|      Date|AveragePrice|pricegroup_mapping(AveragePrice)|
+----------+------------+--------------------------------+
|2015-12-27|        1.33|                        moderate|
|2015-12-20|        1.35|                        moderate|
|2015-12-13|        0.93|                           cheap|
|2015-12-06|        1.08|                        moderate|
|2015-11-29|        1.28|                        moderate|
+----------+------------+--------------------------------+
only showing top 5 rows



Using *'withColumn'*, we can compute values and store them to the new column

In [32]:
new_df = df.withColumn('pricegroup', to_pricegroup(df.AveragePrice))
new_df.select('AveragePrice', 'pricegroup').show(5)

+------------+----------+
|AveragePrice|pricegroup|
+------------+----------+
|        1.33|  moderate|
|        1.35|  moderate|
|        0.93|     cheap|
|        1.08|  moderate|
|        1.28|  moderate|
+------------+----------+
only showing top 5 rows



In [35]:
new_df.show(3)

+---+----------+------------+------------+-------+---------+-----+----------+----------+----------+-----------+------------+----+------+----------+
| Id|      Date|AveragePrice|Total_Volume|   4046|     4225| 4770|Total_Bags|Small_Bags|Large_Bags|XLarge_Bags|        type|year|region|pricegroup|
+---+----------+------------+------------+-------+---------+-----+----------+----------+----------+-----------+------------+----+------+----------+
|  0|2015-12-27|        1.33|    64236.62|1036.74| 54454.85|48.16|   8696.87|   8603.62|     93.25|        0.0|conventional|2015|Albany|  moderate|
|  1|2015-12-20|        1.35|    54876.98| 674.28| 44638.81|58.33|   9505.56|   9408.07|     97.49|        0.0|conventional|2015|Albany|  moderate|
|  2|2015-12-13|        0.93|   118220.22|  794.7|109149.67|130.5|   8145.35|   8042.21|    103.14|        0.0|conventional|2015|Albany|     cheap|
+---+----------+------------+------------+-------+---------+-----+----------+----------+----------+-----------+-

## Pyspark Pandas API
Since release 3.2, Spark provides a Pandas API that enables users to use Pandas-compatible command with just one import (pyspark.pandas).  This lowers learning-curve and enables other features e.g. plotting graph.

In [50]:
!pip uninstall numpy

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/lib/python3.11/dist-packages/numpy-1.26.4.dist-info/*
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libgfortran-040039e1.so.5.0.0
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libopenblas64_p-r0-0cf96a72.3.23.dev.so
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libquadmath-96973f99.so.0.0.0
    /usr/local/lib/python3.11/dist-packages/numpy/*
Proceed (Y/n)? y
  Successfully uninstalled numpy-1.26.4


In [53]:
!pip install numpy==1.26.4 --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 233.3 MB/s eta 0:00:00


In [1]:
import numpy as np

In [2]:
np.__version__

'1.26.4'

In [20]:
import pyspark.pandas as ps

/usr/local/lib/python3.11/dist-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [21]:
pdf = ps.DataFrame(df)

In [22]:
pdf.head()

,Id,Date,AveragePrice,Total_Volume,4046,4225,4770,Total_Bags,Small_Bags,Large_Bags,XLarge_Bags,type,year,region
0,0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


In [23]:
pdf.AveragePrice.plot.box()

In [24]:
counts = pdf.groupby('year').Date.count()

In [25]:
counts

year
2018    1296
2015    5615
2016    5616
2017    5722
Name: Date, dtype: int64

In [26]:
counts.plot.pie()

In [27]:
spark.stop()